```{contents}
```
## Rollout Strategy

A **rollout strategy** in LangGraph defines **how a graph execution is expanded over time**—how decisions, actions, branches, and future states are **explored, evaluated, and committed** during execution.
It is the core mechanism that allows LangGraph to support **planning, simulation, self-refinement, and safe decision-making**.

---

### **1. Intuition: Why Rollouts Exist**

In complex LLM workflows, a single forward pass is often insufficient:

| Problem              | Why Simple Execution Fails          |
| -------------------- | ----------------------------------- |
| Multi-step reasoning | Decisions depend on future outcomes |
| Tool uncertainty     | Tool outputs are unpredictable      |
| Risky actions        | Need to evaluate before committing  |
| Long-horizon tasks   | Errors compound                     |

A rollout allows the system to **simulate future trajectories** before selecting the best path.

> Rollout = *"Try several futures, evaluate them, then commit to one."*

---

### **2. Where Rollout Fits in LangGraph**

```
Current State
     |
   Planner
     |
  ┌──────────┬──────────┬──────────┐
  ↓          ↓          ↓
Trajectory1 Trajectory2 Trajectory3   ← Rollouts
  ↓          ↓          ↓
Evaluate → Select Best → Commit → Continue
```

---

### **3. Core Components of a Rollout**

| Component   | Role                        |
| ----------- | --------------------------- |
| Policy      | Generates candidate actions |
| Environment | Graph + tools               |
| Transition  | Applies action to state     |
| Evaluation  | Scores resulting states     |
| Selection   | Chooses best branch         |
| Commit      | Updates main state          |

---

### **4. Implementing Rollout in LangGraph**

### **State**

```python
class State(TypedDict):
    problem: str
    candidates: list
    chosen: str
```

---

### **Planner Node**

```python
def planner(state):
    actions = llm_generate_actions(state["problem"], k=3)
    return {"candidates": actions}
```

---

### **Rollout Node**

```python
def rollout(state):
    results = []
    for action in state["candidates"]:
        simulated = simulate_action(state, action)
        score = evaluate(simulated)
        results.append((action, score))
    best = max(results, key=lambda x: x[1])[0]
    return {"chosen": best}
```

---

### **Graph**

```python
builder.add_node("plan", planner)
builder.add_node("rollout", rollout)

builder.set_entry_point("plan")
builder.add_edge("plan", "rollout")
builder.add_edge("rollout", END)
```

---

### **5. Variants of Rollout Strategies**

| Strategy             | Description                 | Use Case               |
| -------------------- | --------------------------- | ---------------------- |
| Greedy Rollout       | Pick best immediate outcome | Fast decisions         |
| Beam Rollout         | Keep top-k trajectories     | High-quality planning  |
| Stochastic Rollout   | Randomized exploration      | Creativity, robustness |
| Monte-Carlo Rollout  | Many random simulations     | Risk-sensitive tasks   |
| Hierarchical Rollout | Coarse → fine planning      | Long-horizon goals     |
| ReAct Rollout        | Reason–Act–Observe loops    | Tool-using agents      |

---

### **6. Production Use Cases**

| System             | Role of Rollout         |
| ------------------ | ----------------------- |
| Autonomous agents  | Safe decision selection |
| Trading bots       | Risk-aware execution    |
| Planning engines   | Optimal path selection  |
| Research agents    | Hypothesis exploration  |
| Operations control | Pre-flight validation   |

---

### **7. Safety & Control**

| Mechanism       | Purpose                    |
| --------------- | -------------------------- |
| Recursion limit | Prevent infinite expansion |
| Cost budget     | Control LLM usage          |
| Human gate      | Manual approval            |
| Checkpointing   | Recoverable planning       |
| Audit log       | Explainable decisions      |

```python
graph.invoke(input, config={"recursion_limit": 25})
```

---

### **8. Mental Model**

Rollout in LangGraph turns a static workflow into a **decision search engine**:

> **State → Generate Futures → Simulate → Evaluate → Select → Commit**

This is the foundation of **autonomous, reliable, long-horizon AI systems**.



### Demonstration

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
import random

# ---------------------------
# 1. Define shared state
# ---------------------------
class State(TypedDict):
    goal: str
    candidates: List[str]
    chosen: str

# ---------------------------
# 2. Planner: generate candidate actions
# ---------------------------
def planner(state: State):
    actions = [
        "search web",
        "query database",
        "ask human",
        "run simulation"
    ]
    return {"candidates": actions}

# ---------------------------
# 3. Rollout: simulate + evaluate futures
# ---------------------------
def rollout(state: State):
    scored = []
    
    for action in state["candidates"]:
        # Simulate future outcome (toy model)
        simulated_quality = random.uniform(0, 1)
        scored.append((action, simulated_quality))

    # Select best action
    best_action = max(scored, key=lambda x: x[1])[0]
    return {"chosen": best_action}

# ---------------------------
# 4. Build graph
# ---------------------------
builder = StateGraph(State)

builder.add_node("plan", planner)
builder.add_node("rollout", rollout)

builder.set_entry_point("plan")
builder.add_edge("plan", "rollout")
builder.add_edge("rollout", END)

graph = builder.compile()

# ---------------------------
# 5. Execute
# ---------------------------
result = graph.invoke({"goal": "Solve complex problem"})
print(result)


{'goal': 'Solve complex problem', 'candidates': ['search web', 'query database', 'ask human', 'run simulation'], 'chosen': 'query database'}
